In [9]:
import cv2, dlib
import numpy as np
from imutils import face_utils
from tensorflow.keras.models import load_model
import winsound

IMG_SIZE = (64,56) ## 64 56
B_SIZE = (34, 26)
margin = 80 #95

font_letter = cv2.FONT_HERSHEY_PLAIN
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
model_b = load_model('blinkdetection.h5')
start=0
end=0
blink_per_minute=0
import time
start = time.time()
def detect_blink(eye_img):
    pred_B = model_b.predict(eye_img)
    status = pred_B[0][0]
    status = status*100
    status = round(status,3)
    return  status

   
def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)

    eye_img = gray[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

# main
total=0
prev=0
total1=0
prev1=0
cap = cv2.VideoCapture(0)
# pattern = []
# frames = 10
# pattern_length = 0
frames_to_blink = 6
blinking_frames = 0
while cap.isOpened():
    output = np.zeros((900,820,3), dtype="uint8") #### 900 820 3
    ret, img = cap.read()
    img = cv2.flip(img,flipCode = 1)
    h,w = (112,128)	
    if not ret:
        break

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    

    faces = detector(gray)

    for face in faces:
        shapes = predictor(gray, face)
        

        for n in range(36,42):
            x= shapes.part(n).x
            y = shapes.part(n).y
            next_point = n+1
            if n==41:
                next_point = 36 
            
            x2 = shapes.part(next_point).x
            y2 = shapes.part(next_point).y
            cv2.line(img,(x,y),(x2,y2),(0,69,255),2)

        for n in range(42,48):
            x= shapes.part(n).x
            y = shapes.part(n).y
            next_point = n+1
            if n==47:
                next_point = 42 
            
            x2 = shapes.part(next_point).x
            y2 = shapes.part(next_point).y
            cv2.line(img,(x,y),(x2,y2),(153,0,153),2)
        shapes = face_utils.shape_to_np(shapes)
        #~~~~~~~~~~~~~~~~~56,64 EYE IMAGE~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
        eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])
        #~~~~~~~~~~~~~~~~FOR THE EYE FINAL_WINDOW~~~~~~~~~~~~~~~~~~~~~~#
        eye_img_l_view = cv2.resize(eye_img_l, dsize=(128,112))
        eye_img_l_view = cv2.cvtColor(eye_img_l_view,cv2.COLOR_BGR2RGB)
        eye_img_r_view = cv2.resize(eye_img_r, dsize=(128,112))
        eye_img_r_view = cv2.cvtColor(eye_img_r_view, cv2.COLOR_BGR2RGB)
        #~~~~~~~~~~~~~~~~~FOR THE BLINK DETECTION~~~~~~~~~~~~~~~~~~~~~~~
        eye_blink_left = cv2.resize(eye_img_l.copy(), B_SIZE)
        eye_blink_right = cv2.resize(eye_img_r.copy(), B_SIZE)
        eye_blink_left_i = eye_blink_left.reshape((1, B_SIZE[1], B_SIZE[0], 1)).astype(np.float32) / 255.
        eye_blink_right_i = eye_blink_right.reshape((1, B_SIZE[1], B_SIZE[0], 1)).astype(np.float32) / 255.
        
        status_l = detect_blink(eye_blink_left_i)
        status_r = detect_blink(eye_blink_right_i)
        output = cv2.line(output,(300,240), (300,0),(220,50,220),thickness=2) #  
        
        #~~~~~~~~~~~~~~~~~~~~~~~FINAL_WINDOWS~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        if status_l < 10 :
            if prev==1:
                total+=1
                cv2.putText(output,"---BLINKING----",(250,340), font_letter,2, (200,200,5),2) ## 250,300
                prev=0
        else:
            prev=1
        if status_r < 10 :
            if prev1==1:
                total1+=1
                cv2.putText(output,"---BLINKING----",(250,340), font_letter,2, (200,200,5),2) ## 250,300
                prev1=0
        else:
            prev1=1    
            
        output[0:112, 0:128] = eye_img_l_view # 122,128
#         output[0:112, margin+w:(margin+w)+w] = eye_img_l_view
#         cv2.putText(output,(str(status_l)+"%"),((margin+w),150), font_letter,2, (0,0,255),2)
        output[0:112, 2*margin+2*w:(2*margin+2*w)+w] = eye_img_r_view
#         output[0:112, 3*margin+3*w:(3*margin+3*w)+w] = eye_img_r_view
#         cv2.putText(output, (str(status_l)+"%"),((3*margin+3*w),150), font_letter,2, (0,0,255),2)
        cv2.putText(output, "Blinks {}".format(total1), (400, 160), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0,0,200), 1)
        cv2.putText(output, "Blinks {}".format(total), (8, 160), cv2.FONT_HERSHEY_DUPLEX, 0.7, (255, 128, 0), 1)
        end=time.time()
#       # cv2.putText(output, "Time in sec {}".format(round(end-start,0)), (230, 140), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
        blink_per_minute=(total/(end-start))*60
        blink_per_minute1=(total1/(end-start))*60
        cv2.putText(output, "Blink per minute {}".format(round(blink_per_minute,0)), (8, 200), cv2.FONT_HERSHEY_DUPLEX, 0.7, (255, 128, 0), 1)
        cv2.putText(output, "Blink per minute {}".format(round(blink_per_minute1,0)), (400, 200), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0,0,200), 1)
        cv2.putText(output, "Please romve your spectacles before blink detection", (120, 260), cv2.FONT_HERSHEY_DUPLEX, 0.7, (225,0,0), 1)
        cv2.putText(output, "Designed by Gourav Tiwari", (400, 330), cv2.FONT_HERSHEY_SCRIPT_COMPLEX , 0.5, (0,200,0), 1)
        cv2.putText(output, "Time in sec {}".format(round(end-start,0)), (230, 300), cv2.FONT_HERSHEY_DUPLEX, 0.7, (225,200,200), 1)
        if round(end-start,0)==120.0:
            cap.release()
            cv2.destroyAllWindows()
        output[235+100:715+100, 80:720] = img #  235,715,80:720
        cv2.imshow('result',output)
        
    if cv2.waitKey(1) == ord('q') : 
        break   
cap.release()
cv2.destroyAllWindows() 
print('**************************** result****************************************')
print(' total blink for left_ eye',total,'***************','left eye blink:',blink_per_minute)
print(' total blink for right_ eye',total1,'*************************','left eye blink:',blink_per_minute1)

**************************** result****************************************
 total blink for left_ eye 38 *************** left eye blink: 19.062575504305144
 total blink for right_ eye 35 ************************* left eye blink: 17.557635332912636
